In [32]:
import os
from elasticsearch import Elasticsearch
import pandas as pd

es_host = os.getenv("ELASTICSEARCH_HOST")
es_username = os.getenv("ELASTICSEARCH_USERNAME")
es_password = os.getenv("ELASTICSEARCH_PASSWORD")
es_ca_certs = os.getenv("ELASTICSEARCH_SSL_CERTIFICATEAUTHORITIES")

print(es_ca_certs)

es = Elasticsearch(
    "https://es01:9200",
    basic_auth=("elastic", "ilovethis"),
    ca_certs=es_ca_certs,
    verify_certs=True
)
print(os.access(es_ca_certs, os.R_OK)) 
print(es.info())

/usr/share/elasticsearch/config/certs/ca/ca.crt
True
{'name': 'es01', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'IF9xPO2HTfiDRVNCX23WQQ', 'version': {'number': '8.13.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '617f7b76c4ebcb5a7f1e70d409a99c437c896aea', 'build_date': '2024-04-29T22:05:16.051731935Z', 'build_snapshot': False, 'lucene_version': '9.10.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [33]:

def fetch_latest_logs(index='web-logs', size=10000):
    # Construct the search request to get the latest logs based on the timestamp
    response = es.search(
        index=index,
        body={
            "size": size,  # Include size here within the body
            "query": {"match_all": {}},
            "sort": [{"timestamp": {"order": "desc"}}]
        }
    )
    # Parse the response to get a list of logs
    logs = [hit['_source'] for hit in response['hits']['hits']]
    return logs

# Usage
latest_logs = fetch_latest_logs()


In [34]:
# Fetch logs
logs = fetch_latest_logs()

# Convert logs to a DataFrame for better visualization
logs_df = pd.DataFrame(logs)

# Display the logs DataFrame
logs_df


,timestamp,ip,method,endpoint,response_code,response_time
0,2024-05-08T16:54:19,63.248.59.127,GET,posts,201,727
1,2024-05-08T16:54:18,80.188.147.128,GET,category/blog,200,903
2,2024-05-08T16:54:18,139.246.113.209,PUT,explore/tag,200,402
3,2024-05-08T16:54:18,161.158.212.57,GET,category/category/wp-content,404,673
4,2024-05-08T16:54:18,106.178.13.184,POST,category/search,200,840
...,...,...,...,...,...,...
9995,2024-05-08T15:36:50,101.59.247.238,GET,explore/list,200,135
9996,2024-05-08T15:36:49,47.8.185.56,POST,tag/wp-content/category,404,987
9997,2024-05-08T15:36:49,92.137.180.106,GET,category/tag,404,954
9998,2024-05-08T15:36:49,146.76.228.148,GET,tags/app/category,201,140
